In [1]:
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.cifar10 import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot

2024-05-14 19:58:46.619701: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 19:58:46.619729: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 19:58:46.619749: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-14 19:58:46.625828: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# define the standalone discriminator model
def define_discriminator(in_shape=(32,32,3)):
    model = Sequential()
    # normal
    model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [3]:
# define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
    # foundation for 4x4 image
    n_nodes = 256 * 4 * 4
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4, 4, 256)))
    # upsample to 8x8
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 16x16
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 32x32
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # output layer
    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
    return model

In [4]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [5]:
#THIS ONE
import numpy as np

# load and prepare cifar10 training images
def load_real_samples():
    # Load CIFAR-10 dataset
    (train_images, train_labels), (test_images, test_labels) = load_data()

    # Define class names
    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                'dog', 'frog', 'horse', 'ship', 'truck']

    # Find index of the 'automobile' class
    automobile_class_index = class_names.index('automobile')

    # Filter images and labels to include only 'automobile' class
    automobile_images = train_images[np.where(train_labels == automobile_class_index)[0]]
    # convert from unsigned ints to floats
    X = automobile_images.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return X

In [6]:
# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate ✬real✬ class labels (1)
    y = ones((n_samples, 1))
    return X, y

In [7]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [8]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create ✬fake✬ class labels (0)
    y = zeros((n_samples, 1))
    return X, y

In [9]:
# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
    # scale from [-1,1] to [0,1]
    examples = (examples + 1) / 2.0
    # plot images
    for i in range(n * n):
        # define subplot
        pyplot.subplot(n, n, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(examples[i])
    # save plot to file
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()

In [10]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = 'generator_model_%03d.h5' % (epoch+1)
    g_model.save(filename)

In [11]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=256):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
    # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected ✬real✬ samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            # generate ✬fake✬ examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator✬s error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
            (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
            # evaluate the model performance, sometimes
        if (i+1) % 10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [12]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

2024-05-14 19:58:52.660871: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-14 19:58:53.969723: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-14 19:58:53.970084: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

4/4 [==============================] - 0s 4ms/step
>1, 1/19, d1=0.678, d2=0.711 g=0.689
4/4 [==============================] - 0s 5ms/step
>1, 2/19, d1=0.427, d2=0.858 g=0.636
4/4 [==============================] - 0s 5ms/step
>1, 3/19, d1=0.128, d2=1.518 g=0.716
4/4 [==============================] - 0s 5ms/step
>1, 4/19, d1=0.123, d2=0.605 g=1.254
4/4 [==============================] - 0s 5ms/step
>1, 5/19, d1=0.101, d2=0.574 g=1.001
4/4 [==============================] - 0s 5ms/step
>1, 6/19, d1=0.074, d2=0.709 g=0.771
4/4 [==============================] - 0s 5ms/step
>1, 7/19, d1=0.057, d2=0.654 g=0.767
4/4 [==============================] - 0s 5ms/step
>1, 8/19, d1=0.048, d2=1.937 g=0.743
4/4 [==============================] - 0s 5ms/step
>1, 9/19, d1=0.060, d2=1.314 g=0.909
4/4 [==============================] - 0s 5ms/step
>1, 10/19, d1=0.692, d2=0.626 g=0.782
4/4 [==============================] - 0s 5ms/step
>1, 11/19, d1=0.163, d2=0.630 g=0.796
4/4 [=========================

/home/adi/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 0s 5ms/step
>11, 1/19, d1=0.540, d2=0.732 g=1.796
4/4 [==============================] - 0s 5ms/step
>11, 2/19, d1=0.506, d2=0.661 g=2.020
4/4 [==============================] - 0s 5ms/step
>11, 3/19, d1=0.668, d2=0.606 g=1.990
4/4 [==============================] - 0s 5ms/step
>11, 4/19, d1=0.520, d2=0.729 g=2.565
4/4 [==============================] - 0s 5ms/step
>11, 5/19, d1=0.628, d2=0.819 g=2.514
4/4 [==============================] - 0s 5ms/step
>11, 6/19, d1=0.790, d2=0.514 g=2.320
4/4 [==============================] - 0s 5ms/step
>11, 7/19, d1=0.550, d2=0.561 g=2.231
4/4 [==============================] - 0s 5ms/step
>11, 8/19, d1=0.574, d2=0.812 g=2.025
4/4 [==============================] - 0s 5ms/step
>11, 9/19, d1=0.646, d2=0.618 g=2.008
4/4 [==============================] - 0s 5ms/step
>11, 10/19, d1=0.592, d2=0.787 g=1.868
4/4 [==============================] - 0s 6ms/step
>11, 11/19, d1=0.720, d2=0.763 g=1.296
4/4 [==============

4/4 [==============================] - 0s 5ms/step
>21, 1/19, d1=0.583, d2=0.581 g=1.651
4/4 [==============================] - 0s 7ms/step
>21, 2/19, d1=0.532, d2=0.443 g=1.665
4/4 [==============================] - 0s 6ms/step
>21, 3/19, d1=0.383, d2=0.441 g=1.751
4/4 [==============================] - 0s 6ms/step
>21, 4/19, d1=0.271, d2=0.469 g=1.887
4/4 [==============================] - 0s 6ms/step
>21, 5/19, d1=0.337, d2=0.559 g=2.009
4/4 [==============================] - 0s 6ms/step
>21, 6/19, d1=0.543, d2=0.687 g=2.376
4/4 [==============================] - 0s 6ms/step
>21, 7/19, d1=0.752, d2=0.313 g=2.197
4/4 [==============================] - 0s 6ms/step
>21, 8/19, d1=0.572, d2=0.867 g=2.534
4/4 [==============================] - 0s 6ms/step
>21, 9/19, d1=0.538, d2=0.325 g=2.220
4/4 [==============================] - 0s 5ms/step
>21, 10/19, d1=0.421, d2=0.674 g=2.018
4/4 [==============================] - 0s 6ms/step
>21, 11/19, d1=0.525, d2=0.978 g=2.222
4/4 [==============

...

4/4 [==============================] - 0s 7ms/step
>51, 1/19, d1=0.585, d2=0.472 g=1.740
4/4 [==============================] - 0s 6ms/step
>51, 2/19, d1=0.572, d2=0.736 g=1.741
4/4 [==============================] - 0s 6ms/step
>51, 3/19, d1=0.595, d2=0.458 g=1.965
4/4 [==============================] - 0s 6ms/step
>51, 4/19, d1=0.580, d2=0.465 g=1.788
4/4 [==============================] - 0s 6ms/step
>51, 5/19, d1=0.468, d2=0.703 g=1.863
4/4 [==============================] - 0s 6ms/step
>51, 6/19, d1=0.655, d2=0.568 g=1.627
4/4 [==============================] - 0s 7ms/step
>51, 7/19, d1=0.560, d2=0.533 g=1.552
4/4 [==============================] - 0s 8ms/step
>51, 8/19, d1=0.538, d2=0.627 g=1.704
4/4 [==============================] - 0s 6ms/step
>51, 9/19, d1=0.501, d2=0.420 g=1.853
4/4 [==============================] - 0s 6ms/step
>51, 10/19, d1=0.495, d2=0.437 g=1.655
4/4 [==============================] - 0s 7ms/step
>51, 11/19, d1=0.441, d2=0.646 g=1.722
4/4 [==============

4/4 [==============================] - 0s 7ms/step
>61, 1/19, d1=0.601, d2=0.607 g=1.703
4/4 [==============================] - 0s 6ms/step
>61, 2/19, d1=0.614, d2=0.437 g=1.728
4/4 [==============================] - 0s 6ms/step
>61, 3/19, d1=0.483, d2=0.548 g=1.757
4/4 [==============================] - 0s 6ms/step
>61, 4/19, d1=0.555, d2=0.500 g=1.788
4/4 [==============================] - 0s 7ms/step
>61, 5/19, d1=0.586, d2=0.715 g=1.833
4/4 [==============================] - 0s 6ms/step
>61, 6/19, d1=0.638, d2=0.636 g=1.844
4/4 [==============================] - 0s 6ms/step
>61, 7/19, d1=0.747, d2=0.478 g=1.824
4/4 [==============================] - 0s 6ms/step
>61, 8/19, d1=0.642, d2=0.538 g=1.635
4/4 [==============================] - 0s 7ms/step
>61, 9/19, d1=0.489, d2=0.630 g=1.639
4/4 [==============================] - 0s 6ms/step
>61, 10/19, d1=0.555, d2=0.506 g=1.566
4/4 [==============================] - 0s 6ms/step
>61, 11/19, d1=0.539, d2=0.539 g=1.740
4/4 [==============

4/4 [==============================] - 0s 6ms/step
>71, 1/19, d1=0.492, d2=0.574 g=1.626
4/4 [==============================] - 0s 6ms/step
>71, 2/19, d1=0.643, d2=0.662 g=1.509
4/4 [==============================] - 0s 6ms/step
>71, 3/19, d1=0.585, d2=0.608 g=1.676
4/4 [==============================] - 0s 45ms/step
>71, 4/19, d1=0.678, d2=0.569 g=1.661
4/4 [==============================] - 0s 6ms/step
>71, 5/19, d1=0.610, d2=0.595 g=1.587
4/4 [==============================] - 0s 7ms/step
>71, 6/19, d1=0.532, d2=0.529 g=1.749
4/4 [==============================] - 0s 17ms/step
>71, 7/19, d1=0.624, d2=0.517 g=1.865
4/4 [==============================] - 0s 6ms/step
>71, 8/19, d1=0.545, d2=0.530 g=1.802
4/4 [==============================] - 0s 10ms/step
>71, 9/19, d1=0.586, d2=0.528 g=1.674
4/4 [==============================] - 0s 6ms/step
>71, 10/19, d1=0.598, d2=0.561 g=1.749
4/4 [==============================] - 0s 11ms/step
>71, 11/19, d1=0.707, d2=0.747 g=1.586
4/4 [==========

4/4 [==============================] - 0s 7ms/step
>81, 1/19, d1=0.009, d2=0.007 g=19.000
4/4 [==============================] - 0s 6ms/step
>81, 2/19, d1=0.021, d2=0.045 g=18.915
4/4 [==============================] - 0s 6ms/step
>81, 3/19, d1=0.122, d2=0.058 g=20.965
4/4 [==============================] - 0s 6ms/step
>81, 4/19, d1=0.008, d2=0.034 g=21.001
4/4 [==============================] - 0s 6ms/step
>81, 5/19, d1=0.016, d2=0.006 g=21.535
4/4 [==============================] - 0s 6ms/step
>81, 6/19, d1=0.033, d2=0.014 g=21.688
4/4 [==============================] - 0s 6ms/step
>81, 7/19, d1=0.027, d2=0.019 g=19.689
4/4 [==============================] - 0s 6ms/step
>81, 8/19, d1=0.017, d2=0.022 g=22.587
4/4 [==============================] - 0s 6ms/step
>81, 9/19, d1=0.052, d2=0.002 g=21.370
4/4 [==============================] - 0s 7ms/step
>81, 10/19, d1=0.001, d2=0.012 g=21.576
4/4 [==============================] - 0s 6ms/step
>81, 11/19, d1=0.038, d2=0.021 g=22.272
4/4 [===

4/4 [==============================] - 0s 7ms/step
>91, 1/19, d1=0.000, d2=0.000 g=21.341
4/4 [==============================] - 0s 7ms/step
>91, 2/19, d1=0.000, d2=0.000 g=21.200
4/4 [==============================] - 0s 7ms/step
>91, 3/19, d1=0.090, d2=0.000 g=19.401
4/4 [==============================] - 0s 6ms/step
>91, 4/19, d1=0.001, d2=0.000 g=18.462
4/4 [==============================] - 0s 6ms/step
>91, 5/19, d1=0.004, d2=0.000 g=17.320
4/4 [==============================] - 0s 7ms/step
>91, 6/19, d1=0.000, d2=0.001 g=17.740
4/4 [==============================] - 0s 7ms/step
>91, 7/19, d1=0.010, d2=0.004 g=18.015
4/4 [==============================] - 0s 7ms/step
>91, 8/19, d1=0.000, d2=0.000 g=18.461
4/4 [==============================] - 0s 7ms/step
>91, 9/19, d1=0.000, d2=0.002 g=17.680
4/4 [==============================] - 0s 7ms/step
>91, 10/19, d1=0.000, d2=0.000 g=17.477
4/4 [==============================] - 0s 6ms/step
>91, 11/19, d1=0.058, d2=0.001 g=17.517
4/4 [===